In [1]:
import pandas as pd
import numpy as np
import flirt
import os
import jupyter
import ipywidgets

# 1. Load WESAD and nurses dataset

In [2]:
# load data
df_acc = pd.read_parquet('data-input/dataset_wesad_wrist_acc.parquet')
df_bvp = pd.read_parquet('data-input/dataset_wesad_wrist_bvp.parquet')
df_eda = pd.read_parquet('data-input/dataset_wesad_wrist_eda.parquet')
df_temp = pd.read_parquet('data-input/dataset_wesad_wrist_temp.parquet')

# 2. Function for getting features from FLIRT

In [3]:
def get_features_inner(df, columns_list, prefix, window_length, window_step_size, frequency):
     
    # we need to set a correct datetime index (nanoseconds calculated from frequency)
    # otherwise Flirt will create a wrong timeindex
    ns = '250000000N'
    if frequency == 64:
        ns = '15625000N'
    elif frequency == 32:
        ns = '31250000N'
    time_index = pd.date_range(start=0, periods=len(df), freq=ns)
    df = df.set_index(time_index)
    
    df = df[columns_list]
    df = df.dropna()

    features = flirt.get_acc_features(df,
                                      window_length=window_length, 
                                      window_step_size=window_step_size,
                                      data_frequency=frequency)
    features = features.add_prefix(prefix)
    return features

In [4]:
def get_features(subject, label, df_acc, df_bvp, df_eda, df_temp, window_length, window_step_size):
    
    # calculate features
    acc_features = get_features_inner(df_acc, ['x', 'y', 'z'], 'acc_', window_length, window_step_size, 32)
    bvp_features = get_features_inner(df_bvp, ['BVP'], 'bvp_', window_length, window_step_size, 64)
    eda_features = get_features_inner(df_eda, ['EDA'], 'eda_', window_length, window_step_size, 4)
    temp_features = get_features_inner(df_temp, ['TEMP'], 'temp_', window_length, window_step_size, 4)

    # merge
    res = pd.concat([bvp_features, acc_features, eda_features, temp_features], axis=1)
    
    # add subject and label column
    res['subject'] = subject
    res['label'] = label

    return res

# 3. Calculate features for the whole dataset

In [5]:
df_acc.shape

(882470, 6)

In [6]:
df_acc

,x,y,z,subject,label,session
2972,23.0,-1.0,59.0,10,0,1
2973,24.0,0.0,59.0,10,0,1
2974,25.0,0.0,58.0,10,0,1
2975,23.0,-1.0,57.0,10,0,1
2976,24.0,0.0,59.0,10,0,1
...,...,...,...,...,...,...
80547,59.0,-11.0,17.0,9,1,2
80548,61.0,-13.0,16.0,9,1,2
80549,59.0,-13.0,16.0,9,1,2
80550,59.0,-11.0,16.0,9,1,2


In [7]:
iterlist = [(i, j)
    for i in df_acc.subject.unique()
    for j in df_acc.label.unique()]

In [8]:
def get_all_chunks(df_acc, df_bvp, df_eda, df_temp, subject, label):
    df_acc_chunk = get_chunks(df_acc, subject, label)
    df_bvp_chunk = get_chunks(df_bvp, subject, label)
    df_eda_chunk = get_chunks(df_eda, subject, label)
    df_temp_chunk = get_chunks(df_temp, subject, label)
    return df_acc_chunk, df_bvp_chunk, df_eda_chunk, df_temp_chunk

In [9]:
def get_chunks(df, subject, label):
    df_chunk = df[df['subject'] == subject]
    df_chunk = df_chunk[df_chunk['label'] == label]
    df_chunk = df_chunk.drop(columns=['session', 'subject', 'label'])
    return df_chunk

In [10]:
%%time

result_dfs = []

window_length = 60
window_step_size = 10

# loop over all subject-label combinations (all subjects have 2 sessions with 1 label each)
for (subject, label) in iterlist:
    
    df_acc_chunk, df_bvp_chunk, df_eda_chunk, df_temp_chunk = get_all_chunks(df_acc, df_bvp, df_eda, df_temp, subject, label)
    
    res_df_chunks = get_features(subject, label, df_acc_chunk, df_bvp_chunk, df_eda_chunk, df_temp_chunk, window_length, window_step_size)
    result_dfs.append(res_df_chunks)

res = pd.concat(result_dfs)

ACC features:   0%|          | 0/119 [00:00<?, ?it/s]

Unable to remove memmapped file


ACC features:   0%|          | 0/118 [00:00<?, ?it/s]

Unable to remove memmapped file


ACC features:   0%|          | 0/119 [00:00<?, ?it/s]

...

Unable to remove memmapped file


ACC features:   0%|          | 0/65 [00:00<?, ?it/s]

Unable to remove memmapped file


ACC features:   0%|          | 0/65 [00:00<?, ?it/s]

Unable to remove memmapped file
CPU times: total: 2.17 s
Wall time: 4min 3s


In [11]:
res

,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,temp_l2_n_sign_changes,temp_l2_iqr,temp_l2_iqr_5_95,temp_l2_pct_5,temp_l2_pct_95,temp_l2_entropy,temp_l2_perm_entropy,temp_l2_svd_entropy,subject,label
datetime,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:01:00,0.124318,66.395699,-264.71,272.99,537.70,477.38,1.692827e+07,0.023441,2.820353,145.0,...,0,0.04,0.09,33.25,33.34,5.480639,0.802811,0.003120,10,0
1970-01-01 00:01:10,0.843435,58.168574,-264.71,272.99,537.70,3238.79,1.299569e+07,0.112281,4.464214,149.0,...,0,0.02,0.05,33.29,33.34,5.480639,0.795881,0.003081,10,0
1970-01-01 00:01:20,0.111445,53.332683,-264.71,272.99,537.70,427.95,1.092245e+07,-0.007924,6.198772,153.0,...,0,0.02,0.03,33.31,33.34,5.480639,0.795881,0.002733,10,0
1970-01-01 00:01:30,0.032427,51.826682,-264.71,272.99,537.70,124.52,1.031426e+07,-0.025424,7.110108,158.0,...,0,0.00,0.03,33.31,33.34,5.480639,0.774241,0.002471,10,0
1970-01-01 00:01:40,0.798964,55.666698,-324.07,474.78,798.85,3068.02,1.190177e+07,1.471325,19.029749,160.0,...,0,0.01,0.03,33.31,33.34,5.480639,0.774241,0.002282,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:11:00,-0.071424,61.958522,-755.34,331.38,1086.72,-205.70,1.105593e+07,-3.706059,42.373726,98.0,...,0,0.01,0.03,32.81,32.84,5.187386,0.658744,0.002815,9,1
1970-01-01 00:11:10,-0.538152,20.536645,-87.61,77.53,165.14,-1205.46,9.453772e+05,-0.257402,0.571205,70.0,...,0,0.01,0.03,32.81,32.84,4.934474,0.563856,0.002711,9,1
1970-01-01 00:11:20,-0.865300,20.354221,-87.61,66.83,154.44,-1384.48,6.640689e+05,-0.312787,0.550581,49.0,...,0,0.01,0.01,32.83,32.84,4.595120,0.448864,0.002230,9,1


In [12]:
# handle NANs
res = res.dropna()

In [13]:
res

,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,temp_l2_n_sign_changes,temp_l2_iqr,temp_l2_iqr_5_95,temp_l2_pct_5,temp_l2_pct_95,temp_l2_entropy,temp_l2_perm_entropy,temp_l2_svd_entropy,subject,label
datetime,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:01:00,0.124318,66.395699,-264.71,272.99,537.70,477.38,1.692827e+07,0.023441,2.820353,145.0,...,0,0.04,0.09,33.25,33.34,5.480639,0.802811,0.003120,10,0
1970-01-01 00:01:10,0.843435,58.168574,-264.71,272.99,537.70,3238.79,1.299569e+07,0.112281,4.464214,149.0,...,0,0.02,0.05,33.29,33.34,5.480639,0.795881,0.003081,10,0
1970-01-01 00:01:20,0.111445,53.332683,-264.71,272.99,537.70,427.95,1.092245e+07,-0.007924,6.198772,153.0,...,0,0.02,0.03,33.31,33.34,5.480639,0.795881,0.002733,10,0
1970-01-01 00:01:30,0.032427,51.826682,-264.71,272.99,537.70,124.52,1.031426e+07,-0.025424,7.110108,158.0,...,0,0.00,0.03,33.31,33.34,5.480639,0.774241,0.002471,10,0
1970-01-01 00:01:40,0.798964,55.666698,-324.07,474.78,798.85,3068.02,1.190177e+07,1.471325,19.029749,160.0,...,0,0.01,0.03,33.31,33.34,5.480639,0.774241,0.002282,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:11:00,-0.071424,61.958522,-755.34,331.38,1086.72,-205.70,1.105593e+07,-3.706059,42.373726,98.0,...,0,0.01,0.03,32.81,32.84,5.187386,0.658744,0.002815,9,1
1970-01-01 00:11:10,-0.538152,20.536645,-87.61,77.53,165.14,-1205.46,9.453772e+05,-0.257402,0.571205,70.0,...,0,0.01,0.03,32.81,32.84,4.934474,0.563856,0.002711,9,1
1970-01-01 00:11:20,-0.865300,20.354221,-87.61,66.83,154.44,-1384.48,6.640689e+05,-0.312787,0.550581,49.0,...,0,0.01,0.01,32.83,32.84,4.595120,0.448864,0.002230,9,1


In [14]:
# store as parquet

if not os.path.isdir('data-input'):
    os.makedirs('data-input')

res.to_parquet('data-input/flirt-wesad-acc-bvp-eda-temp-'+str(window_length)+'-'+str(window_step_size)+'.parquet')